KNN

In [ ]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
data=pd.read_csv("./data.csv")
y=data['target']
scaler=StandardScaler()   #标准化
x=data.drop(columns=['id', 'target'])
x=scaler.fit_transform(x)
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)
knn=KNN()
knn.fit(x_train,y_train)
y_pred=knn.predict(x_test)
f1 = f1_score(y_test, y_pred, average='macro')
print(f"F1 Score: {f1}")

SVM

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score
import seaborn as sns
import matplotlib.pyplot as plt

# 读取数据
data = pd.read_csv("./data.csv")

# X = data.iloc[:, :-1]
# y = data.iloc[:, -1]

# 划分训练集和测试集
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100)

Diabetes_data = data.iloc[0:10000, 1:22]

X_train, X_test, y_train, y_test = train_test_split(
    Diabetes_data, data.target.iloc[0:10000], random_state=78, test_size=0.2
)


# 标准化特征值
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 创建SVM模型
svm_model = SVC(kernel='rbf')

# 训练模型
svm_model.fit(X_train, y_train)

# 预测
y_pred = svm_model.predict(X_test)

# 评估模型
accuracy = accuracy_score(y_test, y_pred)
print(f"准确率: {accuracy}")
print("分类报告:")
print(classification_report(y_test, y_pred, zero_division=0))

f1 = f1_score(y_test, y_pred,average="macro")

print(f1)

# 计算并绘制混淆矩阵
conf_matrix = confusion_matrix(y_test, y_pred)
conf_matrix_df = pd.DataFrame(conf_matrix, index=svm_model.classes_, columns=svm_model.classes_)
plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix_df, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predict')
plt.ylabel('Actually')
plt.show()

LightGBM

In [ ]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score

Diabetes = pd.read_csv("./data.csv")

Diabetes_data = Diabetes.iloc[:, 1:22]

x_train, x_test, y_train, y_test = train_test_split(
    Diabetes_data, Diabetes.target, random_state=78, test_size=0.2
)

params_classifier = {
    "boosting_type": "gbdt",
    "objective": "multiclass",
    "num_leaves": 19,
    "learning_rate": 0.3,
    "lambda_l1":0.3,
    "lambda_l2":0.1,
    "n_estimators":1000,
    "feature_fraction": 0.9,
    "bagging_fraction": 0.6,
    "bagging_freq": 1,
    "verbose": -1,
    "num_class": 3,
}

model_classifier = lgb.LGBMClassifier(**params_classifier)

model_classifier.fit(x_train, y_train)

y_pred_classifier = model_classifier.predict(x_test)

f1_classifier = f1_score(y_test, y_pred_classifier,average="macro")

print("f1_score:",f1_classifier)

print("分类报告:")
print(classification_report(y_test, y_pred_classifier, zero_division=0))


XGBoost

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report


#数据集的获取
diabetes=pd.read_csv("./data.csv")
features=['HighBP','HighChol','CholCheck','BMI','Smoker','Stroke','HeartDiseaseorAttack','PhysActivity','Fruits','HvyAlcoholConsump','AnyHealthcare','NoDocbcCost','GenHlth','MentHlth','PhysHlth','DiffWalk','Sex','Age','Education','Income']

#数据集的划分
x_train,x_test,y_train,y_test=train_test_split(diabetes[features],diabetes['target'],random_state=100,test_size=0.3)

#特征工程
transfer=StandardScaler()
x_train=transfer.fit_transform(x_train)
x_test=transfer.fit_transform(x_test)

#设置xgboost的参数
params={'learning_rate':0.3,
        'max_depth':3,
        'objective':'multi:softmax',
        'eval_metric':'mlogloss',
        'num_class':3
}

#将测试集、训练集转化为DMatrix格式
dtrain=xgb.DMatrix(x_train,label=y_train)
dtest=xgb.DMatrix(x_test,label=y_test)

#机器学习
model=xgb.train(params,dtrain,num_boost_round=100,evals=[(dtest,'mlogloss')],early_stopping_rounds=10)

#模型评估
y_pred=model.predict(dtest)
print(y_pred)
f1_score=f1_score(y_test,y_pred,average='macro')
print('f1_score =',f1_score)
accuracy_score=accuracy_score(y_test,y_pred)
print('accuracy_score =',accuracy_score)

决策树

GBDT

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import optuna
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix, make_scorer, f1_score,accuracy_score
from sklearn.model_selection import cross_val_score, KFold

data = pd.read_csv('./data.csv')
x = data.drop('target', axis=1)
y = data['target']

frac_to_use = 0.06
indices = np.random.choice(x.index, size=int(frac_to_use * len(x)), replace=False)
x_small = x.loc[indices]
y_small = y.loc[indices]

x_train, x_test, y_train, y_test = train_test_split(x_small, y_small, test_size=0.2, random_state=142, stratify=y_small)

def optuna_objective(trial):
    max_depth = trial.suggest_int('max_depth', 5, 30)
    learning_rate = trial.suggest_float('learning_rate', 0.001, 0.1, log=True)
    n_estimators = trial.suggest_int('n_estimators', 10, 50)
    subsample = trial.suggest_float('subsample', 0.5, 1.0)
    
    gbc = GradientBoostingClassifier(
        max_depth=max_depth,
        learning_rate=learning_rate,
        n_estimators=n_estimators,
        subsample=subsample,
        random_state=142
    )

    kf = KFold(n_splits=5, shuffle=True, random_state=142)
    cv_scores = cross_val_score(gbc, x_train, y_train, cv=kf, scoring=make_scorer(f1_score, average='macro'))
    mean_cv_score = np.mean(cv_scores)
    
    return mean_cv_score
def optimizer_optuna(n_trials):
    study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())
    study.optimize(optuna_objective, n_trials=n_trials, show_progress_bar=True)
    print(study.best_trial.params, study.best_trial.value)
    return study.best_trial.params, study.best_trial.value
best_params, best_score = optimizer_optuna(25)
best_model = GradientBoostingClassifier(**best_params, random_state=142)
best_model.fit(x_train, y_train)

y_pred = best_model.predict(x_test)
conf_matrix = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='macro')
class_report = classification_report(y_test, y_pred)

print("Confusion Matrix:")
print(conf_matrix)
print("\nAccuracy:")
print(accuracy)
print("\nF1 Score (macro):")
print(f1)
print("\nClassification Report:")
print(class_report)

DecisionTree_1

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score, classification_report

data = pd.read_csv("./data.csv")
x = data.drop('target', axis=1) 
y = data['target']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
clf = DecisionTreeClassifier(random_state=42)
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
f1 = f1_score(y_test, y_pred, average='macro')
print(f"F1-Score : {f1}")
print(classification_report(y_test, y_pred))

DecisionTree_2

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report,f1_score
from scipy.stats import randint
data = pd.read_csv("./data.csv")
x = data.drop('target', axis=1)
y = data['target']
x_train,x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42,stratify=y)
clf = DecisionTreeClassifier(random_state=42)
param_dist = {
    'max_depth': randint(1, 20),
    'min_samples_split': randint(2, 100),
    'min_samples_leaf': randint(1, 100),
    'criterion': ['gini', 'entropy']  
}
random_search = RandomizedSearchCV(clf, param_distributions=param_dist, n_iter=100, cv=5, random_state=42, n_jobs=-1)
random_search.fit(x_train, y_train)
best_clf = random_search.best_estimator_
y_pred = best_clf.predict(x_test)
f1 = f1_score(y_test, y_pred, average='macro') 
print(f"F1-Score : {f1}")
print(classification_report(y_test, y_pred))